# Download dataset from Google Drive

In [ ]:
# !pip install gdown

In [3]:
import gdown

In [ ]:
id = "11oD62_a1nX14uhvpmnlQPmTPQEssuZrc"
gdown.download_folder(id=id, quiet=True, use_cookies=False)

In [12]:
!unzip "/content/bkk_radar_images/*.zip"

'gunzip' is not recognized as an internal or external command,
operable program or batch file.


# Spark Preparation

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

The system cannot find the path specified.


In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!mv spark-3.2.1-bin-hadoop3.2 spark
!pip install -q findspark  

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [2]:
import findspark
findspark.init()

In [3]:
cluster_url = 'local'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master(cluster_url)\
        .appName("App_name")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()
sc = spark.sparkContext

In [ ]:
import glob
from pyspark.sql.functions import from_unixtime, col

data = []

for i in range(1,14):
  folder_path = f'/content/bkk_radar_images_{i}/'
  
  data += glob.glob(folder_path +'*')

rdd = sc.parallelize(data)
rdd2 = rdd.map(lambda x: (x[-14:-4],x[:-14],x[-14:]))

columns = ['DateTime', 'FolderPath', 'FileName']

time_df = spark.createDataFrame(rdd2, columns)
time_df2 = time_df.select(from_unixtime(col('DateTime')), (col('FolderPath')), (col('FileName')))
time_df2 = time_df2.withColumnRenamed('from_unixtime(DateTime, yyyy-MM-dd HH:mm:ss)', 'DateTime')
time_df2 = time_df2.orderBy("DateTime")

In [ ]:
time_df2.printSchema()

root
 |-- DateTime: string (nullable = true)
 |-- FolderPath: string (nullable = true)
 |-- FileName: string (nullable = true)



In [ ]:
time_df2.show(truncate=False)

+-------------------+----------------------------+--------------+
|DateTime           |FolderPath                  |FileName      |
+-------------------+----------------------------+--------------+
|2020-06-18 06:32:34|/content/bkk_radar_images_1/|1592461954.png|
|2020-06-18 06:35:36|/content/bkk_radar_images_1/|1592462136.png|
|2020-06-18 06:46:08|/content/bkk_radar_images_1/|1592462768.png|
|2020-06-18 06:51:08|/content/bkk_radar_images_1/|1592463068.png|
|2020-06-18 06:54:37|/content/bkk_radar_images_1/|1592463277.png|
|2020-06-18 07:01:04|/content/bkk_radar_images_1/|1592463664.png|
|2020-06-18 07:05:02|/content/bkk_radar_images_1/|1592463902.png|
|2020-06-18 07:10:05|/content/bkk_radar_images_1/|1592464205.png|
|2020-06-18 07:18:18|/content/bkk_radar_images_1/|1592464698.png|
|2020-06-18 07:25:53|/content/bkk_radar_images_1/|1592465153.png|
|2020-06-18 07:30:54|/content/bkk_radar_images_1/|1592465454.png|
|2020-06-18 07:40:58|/content/bkk_radar_images_1/|1592466058.png|
|2020-06-1

In [ ]:
time_df2.count()
time_df2.toPandas().to_csv('bkk_all.csv')

86083